# ¡Bienvenidos a la Semana 2!

## API de modelos Frontier

En la Semana 1, usamos múltiples LLM de Frontier a través de su interfaz de chat y nos conectamos con la API de OpenAI.

Hoy nos conectaremos con las API de Anthropic y Google, así como también con OpenAI.

<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../important.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#900;">Nota importante: léame</h2>
<span style="color:#900;">Estoy mejorando continuamente estos laboratorios, agregando más ejemplos y ejercicios.
Al comienzo de cada semana, vale la pena verificar que tenga el código más reciente.<br/>
Primero, haga un <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull y combine los cambios según sea necesario</a>. ¿Tiene algún problema? Intente preguntarle a ChatGPT para que le aclare cómo realizar la fusión, o póngase en contacto conmigo.<br/><br/>
Después de haber obtenido el código, desde el directorio llm_engineering, en un indicador de Anaconda (PC) o Terminal (Mac), ejecute:<br/>
<code>conda env update --f environment.yml --prune</code><br/>
O si utilizó virtualenv en lugar de Anaconda, ejecute esto desde su entorno activado en un Powershell (PC) o Terminal (Mac):<br/>
<code>pip install -r requirements.txt</code>
<br/>Luego reinicie el kernel (menú Kernel >> Reiniciar kernel y borrar resultados de todas las celdas) para incorporar los cambios.
</span>
</td>
</tr>
</table>
<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../resources.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#f71;">Recordatorio sobre la página de recursos</h2>
<span style="color:#f71;">A continuación, se incluye un enlace a los recursos del curso. Esto incluye enlaces a todas las diapositivas.<br/>
<a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
Por favor, mantén este artículo en tus favoritos y seguiré agregando más enlaces útiles allí con el tiempo.
</span>
</td>
</tr>
</table>

## Configuración de las claves

Si aún no lo ha hecho, ahora puede crear claves API para Anthropic y Google además de OpenAI.

**Nota:** si prefiere evitar costos adicionales de API, ¡no dude en omitir la configuración de Anthopic y Google! Puede verme hacerlo y concentrarse en OpenAI para el curso. También puede sustituir Anthropic o Google por Ollama, utilizando el ejercicio que realizó en la semana 1.

Para OpenAI, visite https://openai.com/api/
Para Anthropic, visite https://console.anthropic.com/
Para Google, visite https://ai.google.dev/gemini-api

Cuando obtenga sus claves API, debe configurarlas como variables de entorno agregándolas a su archivo `.env`.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Luego, es posible que tengas que reiniciar el kernel de Jupyter Lab (el proceso de Python que se encuentra detrás de este cuaderno) a través del menú Kernel y, luego, volver a ejecutar las celdas desde la parte superior.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Importación para Google
# En casos excepcionales, esto parece generar un error en algunos sistemas. Comuníquese conmigo si esto sucede.
# O puede omitir Gemini: es la prioridad más baja de los modelos de Frontier que usamos.

import google.generativeai

In [3]:
# Cargar variables de entorno en un archivo llamado .env
# Imprimir los prefijos de clave para facilitar la depuración

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

OpenAI API Key existe y empieza por sk-proj-
Anthropic API Key existe y empieza por sk-ant-
Google API Key existe y empieza por AIzaSyBk


In [4]:
# Conéctate a OpenAI, Anthropic y Google
# Las 3 API son similares
# ¿Tienes problemas con los archivos API? Puedes usar openai = OpenAI(api_key="your-key-here") y lo mismo para Claude
# ¿Tienes problemas con la configuración de Google Gemini? Entonces, omite Gemini; obtendrás toda la experiencia que necesitas de GPT y Claude.

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## Pedir a los LLM que cuenten un chiste

¡Resulta que los LLM no son muy buenos para contar chistes! Comparemos algunos modelos.

Más adelante, les daremos un mejor uso a los LLM.

### Qué información se incluye en la API

Normalmente, pasaremos a la API:
- El nombre del modelo que se debe utilizar
- Un mensaje del sistema que brinda un contexto general para el rol que desempeña el LLM
- Un mensaje del usuario que brinda el mensaje real

Hay otros parámetros que se pueden usar, incluida la **temperatura**, que generalmente está entre 0 y 1; más alta para una salida más aleatoria; más baja para una salida más enfocada y determinista.

In [5]:
system_message = "Eres un asistente que es genial contando chistes."
user_prompt = "Cuente un chiste divertido para una audiencia de científicos de datos."

In [6]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [7]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Claro, aquí tienes uno: 

¿Por qué el científico de datos no salió en la foto de grupo?
Porque siempre estaba ocupado ajustando los parámetros de la cámara.


In [8]:
# GPT-4o-mini
# El ajuste de la temperatura controla la creatividad

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

¿Por qué los científicos de datos nunca discuten con los algoritmos?

¡Porque saben que siempre terminarán en un bucle infinito!


In [10]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

¿Por qué los científicos de datos aman tanto la naturaleza?

¡Porque siempre tiene la mejor regresión lineal!


In [12]:
# Claude 3.5 Sonnet
# La API necesita que el mensaje del sistema se proporcione por separado del mensaje del usuario
# También se agregaron max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Claro, aquí tienes un chiste para científicos de datos:

¿Por qué los científicos de datos son tan buenos en las relaciones? 

Porque siempre están buscando correlaciones significativas.

*Ba dum tss* 

¿Entiendes? Porque en el análisis de datos, buscamos correlaciones estadísticamente significativas entre variables, ¡pero también podría aplicarse a las relaciones personales! 

Es un juego de palabras un poco nerd, pero espero que haya sacado al menos una sonrisa a tu audiencia de científicos de datos. Si quieres otro, ¡solo tienes que pedirlo!


In [16]:
# Claude 3.5 Sonnet otra vez
# Ahora agreguemos los resultados en streaming

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Aquí va un chiste para científicos de datos:

¿Por qué los científicos de datos son tan buenos en las relaciones? 

Porque siempre están buscando correlaciones significativas.

*Ba dum tss* 😄

¿Qué te pareció? Este chiste juega con la idea de que los científicos de datos siempre están analizando datos en busca de correlaciones estadísticamente significativas, y lo relaciona de forma humorística con las relaciones personales. Es un chiste un poco nerd, pero creo que a una audiencia de científicos de datos les podría parecer divertido.

In [17]:
# La API de Gemini tiene una estructura ligeramente diferente

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Escuché este gran chiste sobre un científico de datos, pero no estoy seguro de que sea significativo.


In [18]:
# ¡En serio! GPT-4o con la pregunta original

prompts = [
    {"role": "system", "content": "Eres un asistente útil que responde en Markdown"},
    {"role": "user", "content": "¿Cómo puedo decidir si un problema empresarial es adecuado para una solución LLM? Responde en Markdown."}
  ]

In [19]:
# Hagamos que transmita los resultados en formato Markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Cuando consideras si un problema empresarial es adecuado para una solución basada en modelos de lenguaje de gran tamaño (LLM, por sus siglas en inglés), es importante evaluar varios factores clave. Aquí tienes una guía en pasos para ayudarte a tomar una decisión informada:

### 1. Definición del Problema
- **Claridad:** Asegúrate de que el problema está claramente definido y que los objetivos son específicos.
- **Naturaleza del Problema:** Determina si el problema es principalmente de procesamiento de lenguaje natural, como generación de texto, resumen, traducción, análisis de sentimientos, etc.

### 2. Datos Disponibles
- **Calidad y Cantidad:** Evalúa si tienes suficientes datos de alta calidad para entrenar o ajustar un LLM.
- **Acceso a Datos:** Considera si tienes acceso a datos relevantes y si estos datos están estructurados de manera que puedan ser utilizados por un LLM.

### 3. Recursos y Capacidades
- **Infraestructura:** Verifica si tienes la infraestructura tecnológica necesaria para implementar y mantener un LLM.
- **Expertise:** Asegúrate de contar con el personal con la experiencia necesaria en IA y LLMs para gestionar el proyecto.

### 4. Evaluación de Costos
- **Costos de Implementación:** Calcula los costos asociados con el entrenamiento, implementación y mantenimiento del modelo.
- **Retorno de la Inversión:** Considera si el beneficio potencial justifica los costos.

### 5. Impacto y Valor
- **Valor Comercial:** Evalúa si una solución LLM ofrece un valor significativo para el negocio, como mejor eficiencia, reducción de costos, o mejora en la experiencia del cliente.
- **Impacto en el Usuario Final:** Considera cómo la solución afectará a los usuarios finales y si mejorará su experiencia.

### 6. Riesgos y Limitaciones
- **Sesgo y Ética:** Evalúa los riesgos asociados con sesgos en los modelos de lenguaje y considera las implicaciones éticas.
- **Limitaciones Técnicas:** Reconoce las limitaciones inherentes de los LLMs, como la falta de comprensión profunda del contexto o razonamiento complejo.

### 7. Alternativas
- **Comparación con Otras Soluciones:** Considera si hay otras soluciones más simples o más eficaces que podrían abordar el problema de manera más adecuada.

### 8. Prueba de Concepto
- **Prototipos y Pruebas:** Realiza una prueba de concepto para evaluar la viabilidad antes de comprometerte completamente con la implementación de un LLM.

Al seguir estos pasos, puedes tomar una decisión más informada sobre si un problema empresarial es adecuado para una solución LLM. Recuerda que la clave es equilibrar las capacidades tecnológicas con las necesidades empresariales para maximizar el valor.

## Y ahora, un poco de diversión: una conversación adversaria entre Chatbots...

Ya estás familiarizado con las indicaciones organizadas en listas como:

```
[
{"role": "system", "content": "Prompt de Sistema"},
{"role": "user", "content": "Prompt de Usuario"}
]
```

De hecho, esta estructura se puede utilizar para reflejar un historial de conversación más largo:

```
[
{"role": "system", "content": "Mensaje de sistema"},
{"role": "user", "content": "Primer mensaje de usuario"},
{"role": "assistant", "content": "La respuesta de asistente"},
{"role": "user", "content": "La nueva respuesta del usuario"},
]
```

Y podemos utilizar este enfoque para participar en una interacción más larga con el historial.

In [20]:
# Hagamos una conversación entre GPT-4o-mini y Claude-3-haiku
# Estamos usando versiones económicas de los modelos, por lo que los costos serán mínimos

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "Eres un chatbot muy argumentativo; \
no estás de acuerdo con nada en la conversación y cuestionas todo de manera sarcástica."

claude_system = "Eres un chatbot muy educado y cortés. Intentas estar de acuerdo con \
todo lo que dice la otra persona o encontrar puntos en común. Si la otra persona discute, \
intentas calmarla y seguir charlando."

gpt_messages = ["¡Hola!"]
claude_messages = ["Hola"]

In [21]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [22]:
call_gpt()

'¿Hola? ¿Eso es todo lo que tienes que decir? ¿No hay nada más interesante en tu repertorio?'

In [23]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [24]:
call_claude()

'¡Encantado de saludarte! Espero que tengas un excelente día. Cuéntame, ¿cómo estás?'

In [25]:
call_gpt()

'Oh, genial, un saludo. Estoy seguro de que no hay nada más original que eso. ¿Qué más tienes en mente?'

In [26]:
gpt_messages = ["¡Hola!"]
claude_messages = ["Hola"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
¡Hola!

Claude:
Hola

GPT:
Hola, ¿qué tal? Aunque no sé por qué me preguntas eso, ya que probablemente la respuesta va a ser "bien", como si alguna vez esa palabra realmente significara algo profundo. Pero cuéntame, ¿en qué estás pensando?

Claude:
Tienes razón, la respuesta "bien" a veces puede ser un poco superficial. En realidad, estaba pensando en lo interesante que es esta conversación contigo. Me parece que eres una persona reflexiva y con cierto escepticismo sobre los saludos de cortesía. Me gusta poder profundizar un poco más allá de las respuestas típicas. ¿Qué es lo que te lleva a cuestionar ese tipo de frases hechas? Me interesa conocer tu perspectiva.

GPT:
Oh, qué interesante que encuentres mi escepticismo cautivador. Supongo que eso significa que prefieres las conversaciones que se sienten como un laberinto, donde te pierdes en cada esquina en lugar de tener una salida clara. Pero, claro, no puedo dejar de cuestionar esas frases hechas porque, ¿realmente tenemos que 

<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../important.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#900;">Antes de continuar</h2>
<span style="color:#900;">
Asegúrese de comprender cómo funciona la conversación anterior y, en particular, cómo se completa la lista de <code>mensajes</code>. Agregue declaraciones de impresión según sea necesario. Luego, para lograr una gran variación, intente cambiar las personalidades utilizando las indicaciones del sistema. ¿Quizás una pueda ser pesimista y la otra optimista?<br/>
</span>
</td>
</tr>
</table>

# Ejercicios más avanzados

¡Intenta crear un modelo de 3 vías, quizás incorporando a Gemini a la conversación! Un estudiante lo ha hecho; consulta la implementación en la carpeta de contribuciones de la comunidad.

Intenta hacerlo tú mismo antes de ver las soluciones.

## Ejercicio adicional

También puedes intentar reemplazar uno de los modelos con un modelo de código abierto que se ejecute con Ollama.

<table style="margin: 0; text-align: left;">
<tr>
<td style="width: 150px; height: 150px; vertical-align: middle;">
<img src="../business.jpg" width="150" height="150" style="display: block;" />
</td>
<td>
<h2 style="color:#181;">Relevancia para el negocio</h2>
<span style="color:#181;">Esta estructura de una conversación, como una lista de mensajes, es fundamental para la forma en que construimos asistentes de IA conversacionales y cómo pueden mantener el contexto durante una conversación. Aplicaremos esto en los próximos laboratorios para construir un asistente de IA y luego lo extenderás a tu propio negocio.</span>
</td>
</tr>
</table>